In [1]:
import sympy as smp
from scipy.integrate import odeint,solve_ivp
import numpy as np


In [2]:
t,g,m1,m2,L1,L2, = smp.symbols('t g m1 m2 L1 L2')
theta1 = smp.Function('theta1')
theta1 = theta1(t)
theta2 = smp.Function('theta2')
theta2 = theta2(t)

In [3]:
theta1_d = smp.diff(theta1,t)
theta2_d = smp.diff(theta2,t)
theta1_dd = smp.diff(theta1_d,t)
theta2_dd = smp.diff(theta2_d,t)

In [4]:
x1 = L1*smp.sin(theta1)
x2 = x1+L2*smp.sin(theta2)

y1 = -L1*smp.cos(theta1)
y2 = y1-L2*smp.cos(theta2)


In [5]:
T1 = smp.Rational(1,2) * m1 * (smp.diff(x1,t)**2+smp.diff(y1,t)**2).simplify()
T2 = smp.Rational(1,2) * m2 * (smp.diff(x2,t)**2+smp.diff(y2,t)**2).simplify()
T = T1+T2
V1 = m1*g*y1
V2 = m2*g*y2
V = V1+V2
L=T-V


In [6]:
E1 = smp.diff(L,theta1) - smp.diff(smp.diff(L,theta1_d),t)
E2 = smp.diff(L,theta2) - smp.diff(smp.diff(L,theta2_d),t)

sols = smp.solve([E1, E2], [theta1_dd, theta2_dd], simplify=True)

In [7]:
sols[theta1_dd]

(-L1*m2*sin(2*theta1(t) - 2*theta2(t))*Derivative(theta1(t), t)**2/2 - L2*m2*sin(theta1(t) - theta2(t))*Derivative(theta2(t), t)**2 - g*m1*sin(theta1(t)) - g*m2*sin(theta1(t) - 2*theta2(t))/2 - g*m2*sin(theta1(t))/2)/(L1*(m1 - m2*cos(theta1(t) - theta2(t))**2 + m2))

In [8]:
theta1_dd_f = smp.lambdify((theta1,theta2,theta1_d,theta2_d, m1,m2,L1,L2,g),sols[theta1_dd])
theta2_dd_f = smp.lambdify((theta1,theta2,theta1_d,theta2_d, m1,m2,L1,L2,g),sols[theta2_dd])

In [9]:
def dSdt_svp(t,S,m1,m2,L1,L2,g):
    the1,the2,the1_d,the2_d= S
    return [
    the1_d,
    the2_d,
    theta1_dd_f(the1,the2,the1_d,the2_d,m1,m2,L1,L2,g),
    theta2_dd_f(the1,the2,the1_d,the2_d,m1,m2,L1,L2,g),
    ]

In [10]:
T = 30
t = np.linspace(0,T,500)
y0 = [0,np.pi,3,2]
params = (1,1,1.5,.5,9.8)
sol_ivp = solve_ivp(dSdt_svp,t_span=(0,T),t_eval=t,y0=y0,method='DOP853', args = params)

In [11]:
sol = sol_ivp.y
theta1 = sol[0]
theta2 = sol[1]
theta1_d = sol[2]
theta2_d = sol[3]

In [12]:
%pip install pretty_midi

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pretty_midi

In [14]:
def dp_to_midi(t,
    theta1,
    theta2,
    theta1_d,
    theta2_d,
    out_file="pendulum.mid",
    low_note_1=48,  # C3
    high_note_1=72, # C5
    low_note_2=55,  # G3
    high_note_2=79  # G5
):
    t = np.asarray(t, dtype=float)
    theta1 = np.asarray(theta1, dtype=float)
    theta2 = np.asarray(theta2, dtype=float)
    theta1_d = np.asarray(theta1_d, dtype=float)
    theta2_d = np.asarray(theta2_d, dtype=float)

    #make theta1,theta2 in [-pi,pi]
    theta1 = (theta1+np.pi)%(2*np.pi)-np.pi
    theta2 = (theta2+np.pi)%(2*np.pi)-np.pi

    pm = pretty_midi.PrettyMIDI()
    inst1 = pretty_midi.Instrument(program=0)   # Acoustic Grand Piano
    # Voice 2: strings
    inst2 = pretty_midi.Instrument(program=48)  # String Ensemble 1

    theta_min = -np.pi
    theta_max = np.pi
    pitch1_float = np.interp(theta1,[theta_min,theta_max],[low_note_1,high_note_1])
    pitch2_float = np.interp(theta2,[theta_min,theta_max],[low_note_2,high_note_2])

    eps=1e-5

    theta1_speed = abs(theta1_d)
    theta2_speed = abs(theta2_d)

    max1 = max(theta1_speed)+eps
    max2 = max(theta2_speed)+eps
    vel1_float = 30 + 90 * (theta1_speed / max1)
    vel2_float = 30 + 90 * (theta2_speed / max2)


    for i in range(len(t) - 1):
        start = float(t[i])
        end = float(t[i + 1])
        if end <= start:
            continue  # safety

        # Voice 1 note
        p1 = int(np.clip(round(pitch1_float[i]), 0, 127))
        v1 = int(np.clip(round(vel1_float[i]), 1, 127))

        # Voice 2 note
        p2 = int(np.clip(round(pitch2_float[i]), 0, 127))
        v2 = int(np.clip(round(vel2_float[i]), 1, 127))

        # You can add simple thresholds to avoid super-quiet notes if you want:
        # if v1 > 10:
        inst1.notes.append(
            pretty_midi.Note(
                velocity=v1,
                pitch=p1,
                start=start,
                end=end,
            )
        )
        inst2.notes.append(
            pretty_midi.Note(
                velocity=v2,
                pitch=p2,
                start=start,
                end=end,
            )
        )

    pm.instruments.append(inst1)
    pm.instruments.append(inst2)

    pm.write(out_file)
    print(f"Saved MIDI to {out_file}")


In [15]:
dp_to_midi(t, theta1,theta2,theta1_d,theta2_d)

Saved MIDI to pendulum.mid


In [16]:
!wget https://musical-artifacts.com/artifacts/8/FluidR3_GM.sf2 -O FluidR3_GM.sf2


--2025-12-01 09:09:28--  https://musical-artifacts.com/artifacts/8/FluidR3_GM.sf2
Resolving musical-artifacts.com (musical-artifacts.com)... 104.21.18.50, 172.67.180.102, 2606:4700:3030::6815:1232, ...
Connecting to musical-artifacts.com (musical-artifacts.com)|104.21.18.50|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-12-01 09:09:29 ERROR 404: Not Found.



In [17]:
%pip install pyfluidsynth

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
%pip uninstall -y fluidsynth

Found existing installation: fluidsynth 0.2
Uninstalling fluidsynth-0.2:
  Successfully uninstalled fluidsynth-0.2
Note: you may need to restart the kernel to use updated packages.


In [20]:
import pyfluidsynth
pyfluidsynth.Synth

ModuleNotFoundError: No module named 'pyfluidsynth'

In [ ]:
import fluidsynth

fs = fluidsynth.Synth()
sfid = fs.sfload("FluidR3_GM.sf2")
fs.program_select(0, sfid, 0, 0)

fs.midi_to_audio("pendulum.mid", "pendulum.wav")
fs.delete()


ModuleNotFoundError: No module named 'pyfluidsynth'

In [ ]:
from IPython.display import Audio
Audio("pendulum.wav")


In [21]:
from IPython.display import FileLink
FileLink("pendulum.mid")


/project/rise-portfolio/rise_art_of_science/pendulum.mid